In [20]:
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject

in_path = "geotiff_out/ForUSTree_2018_HighVeg_Density.tif"
out_path = "ForUSTree_2018_HighVeg_Density_50m.tif"

def rastCompress(inputPath, outputPath, pixelLen):
    
    BIG = 1e10  # threshold for "crazy big" values

    with rasterio.open(inputPath) as src:
        src_data = src.read(1).astype("float32")

        # --- 1) CLEAN ---
        # Prefer src.nodata if it exists, but also guard with a threshold
        if src.nodata is not None:
            # float comparison can be finicky; use isclose for safety
            src_data[np.isclose(src_data, src.nodata)] = 0

        # Catch any remaining huge sentinels (e.g., 3.4e38)
        src_data[src_data > BIG] = 0
        src_data[~np.isfinite(src_data)] = 0  # inf/nan -> 0

        # --- 2) COMPUTE NEW SHAPE for ~50m pixels ---
        xres, yres = src.res
        scale_x = float(pixelLen) / xres
        scale_y = float(pixelLen) / yres
        new_width  = int(round(src.width  / scale_x))
        new_height = int(round(src.height / scale_y))

        dst_data = np.zeros((new_height, new_width), dtype="float32")

        # Update transform for the new grid
        dst_transform = src.transform * src.transform.scale(
            src.width / new_width,
            src.height / new_height
        )

        # --- 3) RESAMPLE FROM CLEANED ARRAY ---
        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=src.crs,            # same CRS
            resampling=Resampling.average,
            src_nodata=0,               # critical: treat 0 as nodata in averaging
            dst_nodata=0
        )

        # --- 4) FINAL SAFETY CLEAN (optional but nice) ---
        dst_data[dst_data > BIG] = 0
        dst_data[~np.isfinite(dst_data)] = 0

        # --- 5) WRITE OUT ---
        profile = src.profile.copy()
        profile.update(
            height=new_height,
            width=new_width,
            transform=dst_transform,
            dtype="float32",
            nodata=0
        )

        with rasterio.open(outputPath, "w", **profile) as dst:
            dst.write(dst_data, 1)

    print("Wrote:", outputPath)


In [24]:
rastCompress('geotiff_out/ForUSTree_2018_HighVeg_Height.tif', 'ForUSTree_2018_HighVeg_Height.50m.tif', 50)

Wrote: ForUSTree_2018_HighVeg_Height.50m.tif


In [ ]:
rastCompress('geotiff_out/ForUSTree_2018_HighVeg_TreeCoverage.tif', 'ForUSTree_2018_HighVeg_TreeCoverage', 50)

In [26]:
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject


def rastCompressBinary(inputPath, outputPath, pixelLen, threshold=0.40):
    
    BIG = 1e10  # "crazy big" sentinel threshold

    with rasterio.open(inputPath) as src:
        src_data = src.read(1).astype("float32")

        # --- 1) CLEAN ---
        if src.nodata is not None:
            src_data[np.isclose(src_data, src.nodata)] = 0
        src_data[src_data > BIG] = 0
        src_data[~np.isfinite(src_data)] = 0

        # Force binary 0/1 (handles 0/255, floats, etc.)
        src_bin = (src_data > 0).astype("float32")

        # --- 2) COMPUTE NEW SHAPE for pixelLen ---
        xres, yres = src.res
        scale_x = float(pixelLen) / xres
        scale_y = float(pixelLen) / yres
        new_width  = int(round(src.width  / scale_x))
        new_height = int(round(src.height / scale_y))

        # Destination for fractional coverage (mean of 0/1 = fraction of 1s)
        frac = np.zeros((new_height, new_width), dtype="float32")

        # Update transform for the new grid
        dst_transform = src.transform * src.transform.scale(
            src.width / new_width,
            src.height / new_height
        )

        # --- 3) RESAMPLE: average -> fraction of 1s in each output pixel ---
        reproject(
            source=src_bin,
            destination=frac,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=src.crs,                 # same CRS
            resampling=Resampling.average,
            src_nodata=0,
            dst_nodata=0
        )

        # --- 4) THRESHOLD RULE ---
        out = (frac >= float(threshold)).astype(np.uint8)

        # --- 5) WRITE OUT ---
        profile = src.profile.copy()
        profile.update(
            height=new_height,
            width=new_width,
            transform=dst_transform,
            dtype=rasterio.uint8,
            nodata=0,
            count=1
        )

        with rasterio.open(outputPath, "w", **profile) as dst:
            dst.write(out, 1)

    print("Wrote:", outputPath)


In [28]:
rastCompressBinary('geotiff_out/ForUSTree_2018_HighVeg_TreeCoverage.tif', 'ForUSTree_2018_HighVeg_TreeCoverage.tif', 50)

Wrote: ForUSTree_2018_HighVeg_TreeCoverage.tif


In [31]:

src = rasterio.open('ForUSTree_2018_HighVeg_TreeCoverage.tif')
print(src.res)
print(src.nodata)

data = src.read(1)

(50.00907911802854, 50.01017441860465)
0.0


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(data)
plt.colorbar()
plt.show()


In [18]:
src = rasterio.open("geotiff_out/ForUSTree_2018_HighVeg_Height.tif")
print(src.res)
print(src.nodata)

data = src.read(1)

(2.0, 2.0)
3.3999999521443642e+38
